# Project Drawn Digit

In [ ]:
import numpy as np
from numpy import linalg
import matplotlib.pyplot as plt
import skimage
import skimage.io
import scipy.stats
from skimage.transform import resize
from PIL import ImageTk, Image, ImageDraw
import PIL
from tkinter import *

In [ ]:
# Load in all digits
res = 28
digits = []
for i in range(10):
    digits.append([])
    I = skimage.io.imread("Digits/{}.png".format(i))/255.0
    row = 0
    col = 0
    while row < I.shape[0]:
        col = 0
        while col < I.shape[1]:
            img = I[row:row+res, col:col+res]
            if np.sum(img) > 0:
                digits[i].append(img)
            col += res
        row += res
    print(len(digits[i]), "unique ", i, " digits")

Now let's create our data matrix

In [ ]:
X = []
y = []
for i in range(len(digits)):
    for digit in digits[i]:
        X.append(digit.flatten())
        y.append(i)
X = np.array(X)
y = np.array(y)
print(X.shape)

Finally, we'll subtract off the mean and find the principal components

In [ ]:
mu = np.mean(X, axis=0)


Y = X - mu
A = Y.T.dot(Y)
# v will hold the directions of greatest variance in each column
# By convention, numpy's eigh returns the columns in increasing order of
# eigenvalue, but we want the first few columns to hold the directions 
# of greatest variance, so we flip them
w, v = linalg.eigh(A)
# Sort the columns by descending order of varianc
idx = np.argsort(-w)
w = w[idx]
v = v[:, idx]

In [ ]:
print(w[0])
highest_var = v[:, 0] # Direction of highest variance in "space of digits"
print(np.sqrt(np.sum(highest_var**2)))

highest_var_img = np.reshape(highest_var, (28, 28))

plt.subplot(121)
plt.plot(highest_var)
plt.subplot(122)
plt.imshow(highest_var_img, cmap='magma_r')
plt.colorbar()

In [ ]:
digit = mu - 10*highest_var
plt.imshow(np.reshape(digit, (28, 28)), cmap='gray')
plt.clim([0, 1])

Now draw a digit and project it onto the first few principal components

In [ ]:
n_components = 10 # How many principal components to use
width = 200  # canvas width
height = 200 # canvas height
center = height//2


def project():
    I = np.array(digit_image) # The image that we drew
    I = I[:, :, 0]
    dim = digits[0][0].shape[0]
    I = resize(I, (dim, dim), anti_aliasing=True)
    
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    plt.imshow(I, cmap='gray', vmin=0, vmax=1)
    plt.subplot(122)
    
    ## TODO: Fill code in below to project the digit image
    ## you drew onto the first few directions of v, and add 
    ## the mean back
    v0 = v[:, 0] # Direction of highest variance
    u = I.flatten() - mu
    
    J = mu
    #J = (np.sum(u*v0))*v0 + mu # Hold the result of the projection
    for k in range(n_components):
        J = J + np.dot(u, v[:, k])*v[:, k]
    
    J = np.reshape(J, (28, 28))
    plt.imshow(J, cmap='gray', vmin=0, vmax=1)
    plt.title("{} Components".format(n_components))
    
        
    root.destroy()

def paint(event):
    """
    Paint on the PIL canvas and the Tkinter canvas in parallel
    Draw canvas will be saved, while Tkinter canvas shows
    the user what they are drawing
    """
    bs = 10
    x1, y1 = (event.x - bs), (event.y - bs)
    x2, y2 = (event.x + bs), (event.y + bs)
    canvas.create_oval(x1, y1, x2, y2, fill="black")
    draw.ellipse([x1, y1, x2, y2], fill="#000000")

root = Tk()

# create a tkinter canvas to draw on
canvas = Canvas(root, width=width, height=height, bg='white')
canvas.pack()

# Create a PIL image and a drawer object
digit_image = PIL.Image.new("RGB", (width, height), (255, 255, 255))
draw = ImageDraw.Draw(digit_image)
canvas.pack(expand=YES, fill=BOTH)
canvas.bind("<B1-Motion>", paint)

# add a button to save the image
button=Button(text="project",command=project)
button.pack()

root.mainloop()